# Librerías

In [1]:
!pip install gdown translate langchain sentence-transformers chromadb pymupdf spacy python-decouple llm-templates requests sparqlwrapper gradio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 54.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Tratamiento y consulta PDFs.

In [3]:

import gdown
import os
# Link con archivos sobre la segunda guerra mundial
url = 'https://drive.google.com/drive/u/0/folders/1FJyQ07wq3zB5c0L1Tb14kZjuS3WvyRI-'

# Descarga carpeta
gdown.download_folder(url, quiet=True, output='2da_guerra')

['2da_guerra/2daguerra1.pdf',
 '2da_guerra/2guerra2.pdf',
 '2da_guerra/2guerra3.pdf',
 '2da_guerra/events.csv']

In [4]:
import fitz  # PyMuPDF
import os

# Función para extraer texto de un PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Ruta de la carpeta que contiene los archivos PDF
folder_path = '2da_guerra'

# Lista para almacenar el texto extraído de cada archivo PDF
documents = []

# Recorre la carpeta y extrae texto de cada archivo PDF
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(folder_path, filename)
        text = extract_text_from_pdf(pdf_path)
        documents.append(text)

# Verificar la lista de documentos
for i, doc_text in enumerate(documents):
    print(f"Documento {i+1}:\n{doc_text[:20]}...")  # Muestra los primeros 500 caracteres de cada documento

Documento 1:
http://www.claseshis...
Documento 2:
F. Ayén (2010). “La ...
Documento 3:
 
Geografía e histor...


In [5]:
from langchain.text_splitter import CharacterTextSplitter
import re
all_text = "\n".join(documents)
# Dividir el texto en chunks
splitter = CharacterTextSplitter(separator ="\n",chunk_size=800, chunk_overlap=10)
chunks = splitter.split_text(all_text)

# Función de limpieza
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    cleaned_text = text.replace('\n', ' ').replace('\r', '')
    return cleaned_text

# Limpiar los chunks
cleaned_chunks = [clean_text(chunk) for chunk in chunks]

In [6]:
print(len(cleaned_chunks))

276


In [7]:
from sentence_transformers import SentenceTransformer
# Cargar modelo de embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generar embeddings para los documentos limpios
embeddings = model.encode(cleaned_chunks).tolist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from chromadb import Client

# Configuración inicial de ChromaDB
client = Client()
collection = client.get_or_create_collection("2guerra")

ids_chunck = [f"chunck{i}" for i in range(1, len(cleaned_chunks)+1)]

collection.add(
    documents=cleaned_chunks,
    ids=ids_chunck,
    embeddings=embeddings
)

In [9]:
# Función para obtener contexto del PDF
def get_context_from_pdf(query):
    # Generar embeddings para la consulta
    embedding_consulta = model.encode([query]).tolist()

    # Realizar la consulta en ChromaDB
    results = collection.query(
        query_embeddings=embedding_consulta,  # Aquí pasamos el embedding de la consulta
        n_results=1
    )

    # Si hay resultados, devolver el contexto del documento más cercano
    if results['documents']:
        context = results['documents'][0]
    else:
        context = "No se encontró contexto relevante."

    return context

# Tratamiento y consulta de base de dato tabular.

In [10]:
import pandas as pd
import spacy
from translate import Translator as AzureTranslator

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')

# DataFrame de ejemplo
df = pd.read_csv('2da_guerra/events.csv')

# Función para extraer el nombre del evento usando NER
def extract_relevant_info(query):
    doc = nlp(query)
    extracted_info = []
    for ent in doc.ents:
        if ent.label_ in ["LOC", "MISC"]:  # LOC para ubicaciones, MISC para eventos
            extracted_info.append(ent.text)
    return extracted_info if extracted_info else [query]  # Devolver la consulta original si no se encuentra ninguna entidad relevante

# Función para traducir el nombre del evento al inglés
def translate_event_names(event_names):
    translator = AzureTranslator(to_lang='en', from_lang='es')
    translated_names = [translator.translate(name) for name in event_names]
    return translated_names

# Función para obtener contexto del CSV usando nombres de eventos traducidos
def get_context_from_csv(query):
    relevant_info = extract_relevant_info(query)
    translated_names = translate_event_names(relevant_info)

    results = df[df.apply(
        lambda row: any(name.lower() in row['Event'].lower() for name in translated_names),
        axis=1
    )]

    # Si se encuentran resultados, crear el contexto
    if not results.empty:
        context = ""
        for _, row in results.iterrows():
            event = row["Event"]
            date = row["Date"]
            context += f"Evento: {event}, Fecha: {date}\n"
    else:
        context = "No se encontró información relevante en el CSV."

    return context

# Tratamiento y consulta de base de datos de grafos.

In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Función para extraer el nombre del personaje usando NER
def extract_person_name(query):
    doc = nlp(query)
    for ent in doc.ents:
        if ent.label_ == "PER":
            return ent.text
    return query  # Devolver la consulta original si no se encuentra ningún nombre

def get_context_from_wikidata(query):
    person_name = extract_person_name(query)

    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Crear la consulta SPARQL
    sparql_query = f"""
    SELECT ?item ?itemLabel ?description WHERE {{
      ?item ?label "{person_name}"@en.
      ?item wdt:P31 wd:Q5.
      OPTIONAL {{ ?item schema:description ?description. FILTER (LANG(?description) = "en") }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }} LIMIT 1
    """

    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    # Procesar los resultados
    context = []
    for result in results["results"]["bindings"]:
        item_label = result["itemLabel"]["value"]
        description = result.get("description", {}).get("value", "No description available")
        context.append(f"{item_label}: {description}")

    return context

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


# Clasificadores

In [12]:
from transformers import pipeline
# Cargar el pipeline de clasificación
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# Función para clasificar consultas
def classify_query_with_llm(query):
    # Definir las etiquetas candidatas con descripciones contextuales
    candidate_labels = {
        "csv": "Consultas sobre fechas de eventos históricos específicos, por ejemplo '¿Cuándo ocurrió la batalla de Stalingrado?'",
        "wikidata": "Consultas sobre personajes históricos importantes, por ejemplo '¿Quién fue Anne Frank?'",
        "pdf": "Consultas generales sobre contextos históricos y eventos, por ejemplo 'Explícame las consecuencias económicas de la Segunda Guerra Mundial.'"
    }

    # Realizar la clasificación
    result = classifier(query, list(candidate_labels.values()))

    # Obtener la etiqueta con la puntuación más alta
    best_label = result['labels'][0].lower()
    confidence = result['scores'][0]

    return best_label, confidence

# Ejemplos de consultas para probar la clasificación
queries = [
    "¿Qué país lanzó la bomba atómica sobre Hiroshima y Nagasaki?",
    "¿Quién fue Adolf Hitler?",
    "¿Cuándo ocurrió la batalla de Midway?",
    "Explícame cómo afectó la guerra a la economía de Alemania.",
    "¿Cuál fue la estrategia militar utilizada en la Batalla de las Ardenas?"
]

# Clasificar y mostrar resultados
for query in queries:
    label, confidence = classify_query_with_llm(query)
    print(f"Consulta: {query}")
    print(f"Clasificación con LLM: {label} (Confianza: {confidence:.2f})")
    print("-----------------------")

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Consulta: ¿Qué país lanzó la bomba atómica sobre Hiroshima y Nagasaki?
Clasificación con LLM: consultas sobre fechas de eventos históricos específicos, por ejemplo '¿cuándo ocurrió la batalla de stalingrado?' (Confianza: 0.77)
-----------------------
Consulta: ¿Quién fue Adolf Hitler?
Clasificación con LLM: consultas generales sobre contextos históricos y eventos, por ejemplo 'explícame las consecuencias económicas de la segunda guerra mundial.' (Confianza: 0.60)
-----------------------
Consulta: ¿Cuándo ocurrió la batalla de Midway?
Clasificación con LLM: consultas sobre fechas de eventos históricos específicos, por ejemplo '¿cuándo ocurrió la batalla de stalingrado?' (Confianza: 0.54)
-----------------------
Consulta: Explícame cómo afectó la guerra a la economía de Alemania.
Clasificación con LLM: consultas generales sobre contextos históricos y eventos, por ejemplo 'explícame las consecuencias económicas de la segunda guerra mundial.' (Confianza: 0.91)
-----------------------
Consu

In [13]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Datos de ejemplos
examples = {
    "csv": [
        "¿Cuándo ocurrió la Batalla de Stalingrado?",
        "¿Qué evento histórico sucedió el 6 de agosto de 1945?",
        "¿Cuál fue la fecha del inicio de la Segunda Guerra Mundial?",
        "¿Cuándo se firmó el Tratado de Versalles?",
        "¿Qué evento marcó el final de la Guerra Fría?",
        "¿Cuál fue la fecha de la caída del Muro de Berlín?"
        "¿Cuándo ocurrió la Batalla del Alamein?",
        "¿Qué fecha marca el inicio de la Guerra de Vietnam?",
        "¿Cuándo se estableció la Liga de las Naciones?",
        "¿Qué evento histórico sucedió el 7 de diciembre de 1941?",
        "¿Cuál fue la fecha del Armisticio de la Primera Guerra Mundial?",
        "¿Cuándo se celebró el primer desfile del Día de la Victoria en Rusia?"
    ],
    "wikidata": [
        "¿Quién fue Winston Churchill?",
        "¿Dónde nació Mahatma Gandhi?",
        "¿Cuál fue el descubrimiento más famoso de Isaac Newton?",
        "¿Quién fue Albert Einstein?",
        "¿Dónde nació Leonardo da Vinci?",
        "¿Qué personaje histórico fue conocido como el 'Rey Sol'?"
         "¿Quién fue Cleopatra?",
        "¿Dónde nació Karl Marx?",
        "¿Cuál fue el descubrimiento más famoso de Marie Curie?",
        "¿Quién fue Gengis Kan?",
        "¿Dónde nació Charles Darwin?",
        "¿Qué personaje histórico fue conocido como el 'Rey Sol'?"
    ],
    "pdf": [
        "Explícame cómo afectó la guerra a la economía de Alemania.",
        "¿Cuál fue la estrategia militar utilizada en la Batalla de las Ardenas?",
        "¿Cómo cambió la vida cotidiana durante la Segunda Guerra Mundial?",
        "¿Qué avances tecnológicos surgieron durante la Segunda Guerra Mundial?",
        "¿Cuál fue el impacto cultural de la Guerra Fría?",
        "¿Cómo influyó la Guerra Civil Española en la política europea?"
        "Explícame cómo afectó la Guerra Fría a América Latina.",
        "¿Cuál fue la estrategia de combate utilizada en la Batalla de Verdún?",
        "¿Cómo afectó la Segunda Guerra Mundial a la industria cinematográfica?",
        "¿Qué innovaciones tecnológicas surgieron durante la Revolución Industrial?",
        "¿Cuál fue el impacto cultural de la Revolución Francesa?",
        "¿Cómo influyó la Revolución Mexicana en la política internacional?"
    ]
}



# Cargar el modelo SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Preprocesar ejemplos para calcular embeddings
example_embeddings = {}
for category, queries in examples.items():
    embeddings = model.encode(queries)
    example_embeddings[category] = {
        "queries": queries,
        "embeddings": embeddings
    }

# Función para clasificar una consulta dada
def classify_query_with_examples(query, example_embeddings):
    # Calcular el embedding de la consulta
    query_embedding = model.encode(query)

    # Calcular la similitud de coseno con cada grupo de ejemplos
    similarities = {}
    for category, data in example_embeddings.items():
        example_vectors = data["embeddings"]
        sim_scores = util.pytorch_cos_sim(query_embedding, example_vectors)[0]
        avg_score = np.mean(sim_scores.cpu().numpy())  # Calcular la media sobre los valores de similitud
        similarities[category] = avg_score

    # Determinar la clase con la similitud más alta
    best_category = max(similarities, key=similarities.get)
    confidence = similarities[best_category]

    return best_category, confidence

# Ejemplos de consultas para probar la clasificación
queries = [
    "¿Cuál fue el rol de la Resistencia Francesa durante la Segunda Guerra Mundial?",
    "¿Qué impacto tuvo la Segunda Guerra Mundial en la economía de Alemania?",
    "¿En que fecha ocurrio la Batalla de El Alamein ?",
    "¿Cuando fue la Batalla de Kursk ?",
    "¿Quién fue Klaus Barbie ?",
    "Explícame cómo afectó la Batalla de Midway a la estrategia naval japonesa en el Pacífico.",
    "¿Dónde nacio Benito Mussolini?",
    "¿Cuál fue la estrategia de evacuación de Dunkerque y qué papel jugó en la guerra?",
    "¿Qué eventos llevaron al bombardeo de Pearl Harbor y cómo cambió la estrategia de Estados Unidos?",
]

# Clasificar y mostrar resultados
for query in queries:
    label, confidence = classify_query_with_examples(query, example_embeddings)
    print(f"Consulta: {query}")
    print(f"Clasificación con Ejemplos: {label} (Confianza: {confidence:.2f})")
    print("-----------------------")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Consulta: ¿Cuál fue el rol de la Resistencia Francesa durante la Segunda Guerra Mundial?
Clasificación con Ejemplos: pdf (Confianza: 0.49)
-----------------------
Consulta: ¿Qué impacto tuvo la Segunda Guerra Mundial en la economía de Alemania?
Clasificación con Ejemplos: pdf (Confianza: 0.54)
-----------------------
Consulta: ¿En que fecha ocurrio la Batalla de El Alamein ?
Clasificación con Ejemplos: csv (Confianza: 0.49)
-----------------------
Consulta: ¿Cuando fue la Batalla de Kursk ?
Clasificación con Ejemplos: csv (Confianza: 0.56)
-----------------------
Consulta: ¿Quién fue Klaus Barbie ?
Clasificación con Ejemplos: wikidata (Confianza: 0.38)
-----------------------
Consulta: Explícame cómo afectó la Batalla de Midway a la estrategia naval japonesa en el Pacífico.
Clasificación con Ejemplos: pdf (Confianza: 0.34)
-----------------------
Consulta: ¿Dónde nacio Benito Mussolini?
Clasificación con Ejemplos: wikidata (Confianza: 0.42)
-----------------------
Consulta: ¿Cuál fue l

A diferencia del clasificador 'zero shot' el de ejemplos con comparación de embeddings es mas eficaz.

# Prueba de clasificador y contexto.

In [14]:
# Función principal para probar las consultas
def prueba_query(query):
    try:
        # Clasificar la consulta
        best_category, confidence = classify_query_with_examples(query,example_embeddings)

        # Imprimir la categoría clasificada y la confianza
        print(f"Consulta: '{query}'\nCategoría clasificada: {best_category} (Confianza: {confidence})")

        # Obtener el contexto según la clasificación
        if best_category == "pdf":
            context = get_context_from_pdf(query)
        elif best_category == "wikidata":
            context = get_context_from_wikidata(query)
        elif best_category == "csv":
            context = get_context_from_csv(query)
        else:
            context = "No se pudo determinar la fuente de datos adecuada para responder la consulta."

        # Imprimir el contexto para la consulta
        print(f"Contexto para '{query}':\n{context}\n")

    except Exception as e:
        print(f"An error occurred: {e}")

# Pruebas de las consultas
queries = [
    "¿Cuándo ocurrió el estallido de la bomba en Nagasaki?",
    "¿Por qué entró Estados Unidos a la guerra?",
    "¿Cuándo fue la conferencia naval de Londres?",
    "¿Quién fue Benito Mussolini?",
    '¿Que armamento utilizo Alemania?',
    '¿Quien fue Franklin Roosevelt?'
]

for query in queries:
    prueba_query(query)

Consulta: '¿Cuándo ocurrió el estallido de la bomba en Nagasaki?'
Categoría clasificada: csv (Confianza: 0.5697488784790039)
Contexto para '¿Cuándo ocurrió el estallido de la bomba en Nagasaki?':
Evento: Atomic Bombing of Hiroshima and Nagasaki, Fecha: 6 Aug 1945 - 9 Aug 1945


Consulta: '¿Por qué entró Estados Unidos a la guerra?'
Categoría clasificada: pdf (Confianza: 0.3721785545349121)
Contexto para '¿Por qué entró Estados Unidos a la guerra?':
['hicieron que el conflicto se convirtiera en una guerra mundial. Alemania invadió la URSS en junio,  Japón atacó la base naval norteamericana de Pearl Harbor en diciembre. La URSS y EEUU se  unieron a Gran Bretaña en el bando de los Aliados. A partir de ese momento, la guerra cambió  radicalmente de signo.   DECLARACIÓN DE LAS NACIONES UNIDAS     resistencia francesa no debe apagarse y no se  apagará. ”   Charles de Gaulle Londres, 18 de junio de 1940     Geografía e historia 4º ESO |  7      La segunda guerra mundial   y sus consecuencias 

# Modelo LLM

In [15]:
import requests
from decouple import config
from google.colab import userdata

from llm_templates import Formatter, Conversation

# Función para aplicar el template de chat usando la librería llm-templates
def zephyr_chat_template(messages, add_generation_prompt=True):
    formatter = Formatter()
    conversation = Conversation(model='zephyr', messages=messages)
    return formatter.render(conversation, add_assistant_prompt=add_generation_prompt)

# Aquí hacemos la llamada al modelo de generación de texto
def generate_answer(prompt: str, max_new_tokens: int = 768) -> str:
    try:
        # Tu clave API de Hugging Face.
        api_key = config('HUGGINGFACE_TOKEN', userdata.get('token_pln'))

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Verificar el código de estado de la respuesta
        if response.status_code != 200:
            raise Exception(f"Request failed with status code {response.status_code}. Response: {response.text}")

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Función para preparar el prompt en estilo QA
def prepare_prompt(query_str: str, context_str: str):
    TEXT_QA_PROMPT_TMPL = (
        " Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos. Por favor, responde siempre en español. \n"
        "La información de contexto es la siguiente:\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta en español.\n"
        "Pregunta: {query_str}\n"
        " Respuesta: "
    )

    messages = [
        {"role": "system", "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos. Por favor, responde siempre en español."},
        {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_str=context_str, query_str=query_str)}
    ]

    final_prompt = zephyr_chat_template(messages)
    return final_prompt



# Chatbot

In [17]:
import gradio as gr
# Usa una interfaz de ChatInterface para el chat

def chat_response(query, history):
    try:
        # Clasificar la consulta utilizando ejemplos y embeddings
        best_category, confidence = classify_query_with_examples(query, example_embeddings)

        # Obtener el contexto según la clasificación
        if best_category == "pdf":
            context = get_context_from_pdf(query)
        elif best_category == "wikidata":
            context = get_context_from_wikidata(query)
        elif best_category == "csv":
            context = get_context_from_csv(query)
        else:
            context = "No se pudo determinar la fuente de datos adecuada para responder la consulta."

        # Preparar el prompt con el contexto
        prompt = prepare_prompt(query, context)

        # Generar la respuesta
        respuesta = generate_answer(prompt)
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")
        return "Lo siento, ocurrió un error al procesar tu consulta."

# Configura la interfaz de chat
demo = gr.ChatInterface(
    fn=chat_response,
    examples=[
        "¿Cuándo ocurrió el estallido de la bomba en Nagasaki?",
        "¿Por qué entró Estados Unidos a la guerra?",
        "¿Cuándo fue la conferencia naval de Londres?",
        "¿Quién fue Benito Mussolini?",
        '¿Que armamento utilizo Alemania?',
        '¿Quien fue Franklin Roosevelt?'
    ],
    title='Chatbot Especialista en la Segunda Guerra Mundial ',
    chatbot=gr.Chatbot(value=[(None, "¡Hola! Soy un especialista en la Segunda Guerra Mundial. Puedes preguntarme cualquier cosa relacionada con este periodo histórico.")])
)

# Lanza la interfaz de chat
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a7cd268851a90059cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a7cd268851a90059cf.gradio.live
